In [16]:
class GridworldRow():
    def __init__(self, size=5):
        self.size = size

        self.nS = size
        self.nA = 2

        self.MAX_X = size-1

        P = {}

        for s in range(self.nS):
            dynamics_s = {}

            for a in range(self.nA):
                s_prime_list = []

                p = 1 if s != 0 and s != self.MAX_X else 0

                if(a == 0):
                    s_prime = max(0, s-1)
                else:
                    s_prime = min(self.MAX_X, s+1)
                if (s_prime == 0):
                    reward = -100
                    done = True
                elif (s_prime == self.MAX_X):
                    reward = 10
                    done = True
                else:
                    reward=0
                    done = False

                s_prime_list.append((p, s_prime, reward, done))
                dynamics_s.update({a:s_prime_list})
            
            P.update({s: dynamics_s})
        self.P = P

In [17]:
env = GridworldRow()
env.P

{0: {0: [(0, 0, -100, True)], 1: [(0, 1, 0, False)]},
 1: {0: [(1, 0, -100, True)], 1: [(1, 2, 0, False)]},
 2: {0: [(1, 1, 0, False)], 1: [(1, 3, 0, False)]},
 3: {0: [(1, 2, 0, False)], 1: [(1, 4, 10, True)]},
 4: {0: [(0, 3, 0, False)], 1: [(0, 4, 10, True)]}}

In [18]:
import numpy as np
env=GridworldRow()
pi = np.ones([env.nS, env.nA]) * 0.5
V = np.zeros([env.nS, 1])

gamma =0.99 #facteur de remise du return
theta = 0.00001 #seuil de similitude requis pour stopper les updates

In [19]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    for (p_sprime, sPrime, r_ss_a, done) in env.P[s][a]:
        q += p_sprime * (r_ss_a + gamma * V[sPrime])
    return q

In [20]:
# Boucle d'update de l'algo de policy evaluation
i = 0
while True:
    i+=1
    delta = 0

    for s in range(env.nS):
        V_new = 0
    
        for a in range(env.nA):
            prob_a = pi[s][a]
            q_s_a = compute_q_value_for_s_a(env, V, s, a, gamma)
        
            V_new+= prob_a * q_s_a
        
        delta = max(delta, np.abs (V_new - V[s]))
        V[s] =V_new
    if(delta < theta):
        print("Terminé après " + str(i) +" itérations")
        break

Terminé après 22 itérations


In [21]:
V

array([[  0.        ],
       [-71.62196676],
       [-43.6807471 ],
       [-16.62196981],
       [  0.        ]])

In [22]:
# Policy Improvement
import numpy as np
env = GridworldRow()
pi = np.ones([env.nS, env.nA]) * 0.50
gamma = 0.99 #facteur de remise du return
V = np.array([[0], [-71.6], [-43.6], [-16.6], [0]])
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0

    for (p_sPrime, sPrime, r_ss_a, done) in env.P[s] [a]:
        q += p_sPrime * (r_ss_a+ gamma + V[sPrime])
        return q

In [23]:
for s in range(env.nS):
    q_s = np.zeros([env.nA, 1])

    for a in range(env.nA):
        q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)

    best_a = np.argmax(q_s)
    pi[s] = np.eye(env.nA)[best_a]

In [24]:
pi # les 1 de la collone de droite sont les actions qui font gagner pour arriver au reward

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [42]:
#Policy Iteration algorithm
import numpy as np
env = GridworldRow()
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0

    for (p_sPrime, sPrime, r_ss_a, done) in env.P[s][a]:
        q += p_sPrime * (r_ss_a+ gamma + V[sPrime])

    return q

In [46]:
def evaluate_policy(env, pi, V, gamma, theta):
    V_updated = np.copy(V)
    improved = True
    print(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            
            V_new = 0

            for a in range(env.nA):
                prob_a = pi[s][a]
                q_s_a = compute_q_value_for_s_a(env, V_updated, s, a, gamma)

                V_new +=  prob_a * q_s_a

            delta = max(delta, np.abs(V_new - V_updated[s]))
            V_updated[s] = V_new

        if(delta < theta):
            break

    if (np.array_equal(V, V_updated)):
        improved = False

    return V_updated, improved



In [40]:
def improve_policy(env, pi, V, gamma):
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])

        for a in range(env.nA):
            q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)

        best_a = np.argmax(q_s)
        pi[s] = np.eye(env.nA)[best_a]
    
    return pi

In [31]:
pi = np.ones([env.nS, env.nA]) * 0.5
V = np.zeros([env.nS, 1]) 

gamma = 0.99 #facteur de return
theta = 0.00001 #seuil de similitude requis pour stopper les updates


In [41]:
i = 0
while True:
    i+=1

    V, improved = evaluate_policy(env, pi, V, gamma, theta)
    pi = improve_policy(env, pi, V, gamma)

    if(improved == False):
        print("Terminé après" + str(i) + "itérations.")
        break

KeyboardInterrupt: 

In [ ]:
pi